In [ ]:
import polars as pl
import json

QUERIES_FOLDER = "../1.query/extended_demography/"
OUTPUT_FOLDER = "../2.rawDataset/extended_demography/"
SERVER = 'SVM-ICCA-REP'
DATABASE = 'CISReportingDB'

In [ ]:
encounters = pl.read_parquet('../4.preparedDatasets/prepared_dataset_2024-11-08.parquet').select("encounterId", "utcInTime")

In [ ]:
conn_uri = f'mssql://@{SERVER}/{DATABASE}?trusted_connection=true'

query = open(QUERIES_FOLDER + 'generic_query.sql').read()

In [ ]:
with open(QUERIES_FOLDER + 'features.json', 'r', encoding='utf-8') as f :
    variables_json = json.load(f)

## Génération des datasets

### Automatisée via JSON

In [ ]:
for variable in variables_json :
    propname = variables_json[variable]['dictionary_propname']
    table = variables_json[variable]['table']
    feature = variables_json[variable]
    query_formatted = query.format(
        dictionaryPropName = f"'{propname}'",
        feature = f"'{variable}'",
        feature_table = table
    )
    try :
        feature_data = pl.read_database_uri(query_formatted, conn_uri, engine='connectorx')
        print(f'Extraction {variable} : OK')
    except :
        raise

    with_time_delta = feature_data.with_columns(
        pl.col('encounterId').cast(pl.String)
    ).join(
        encounters, on='encounterId', how="inner"
    ).with_columns(
        (((pl.col('utcChartTime') - pl.col('utcInTime')).dt.total_minutes())/60).alias('delta_inTime_hours')
    ).sort('encounterId', 'utcChartTime')
    try :
        with_time_delta.write_parquet(f'{OUTPUT_FOLDER}{variable}.parquet')
        print(f'Sauvegarde {variable} : OK  / Dataset shape : {with_time_delta.shape}')
    except :
        raise

In [ ]:
for variable in variables_json :
    print(variable)

In [ ]:
feature_data = pl.read_database_uri(query, conn_uri, engine='connectorx')

In [ ]:
with_time_delta = feature_data.with_columns(
    pl.col('encounterId').cast(pl.String)
).join(
    encounters, on='encounterId', how="inner"
    ).with_columns(
        (((pl.col('utcChartTime') - pl.col('utcInTime')).dt.total_minutes())/60).alias('delta_inTime_hours')
    ).sort('encounterId', 'utcChartTime')


In [ ]:
with_time_delta.write_parquet('../2.rawDataset/dynamic_features/map_invasive.parquet')